# Seseme CSM Fine-Tuning

This notebook demonstrates how to fine-tune the Seseme CSM model using the Hugging Face `transformers` library.

## 1. Setup

Connect to the Hugging Face Hub and install the dataset. The dataset used is in `parquet` format, which is efficient for large datasets.

In [6]:
from huggingface_hub import login
import os

# initialize login token
login()

In [7]:
!git clone https://huggingface.co/datasets/MrDragonFox/Elise

Cloning into 'Elise'...
remote: Enumerating objects: 17, done.
remote: Total 17 (delta 0), reused 0 (delta 0), pack-reused 17 (from 1)
Unpacking objects: 100% (17/17), 5.95 KiB | 1.98 MiB/s, done.


## 2. Preprocessing

### 2.1 Load The Model

In [8]:
from transformers import CsmForConditionalGeneration, AutoProcessor, Trainer, TrainingArguments
from datasets import load_dataset, Audio
import torch
import numpy as np
from tqdm import tqdm
import os

model_id = "sesame/csm-1b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor
processor = AutoProcessor.from_pretrained(model_id)
model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)
model.train()
model.codec_model.eval()  # Keep codec model in eval mode during training

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

...

transformers-00001-of-00002.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

transformers-00002-of-00002.safetensors:   0%|          | 0.00/2.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

MimiModel(
  (encoder): MimiEncoder(
    (layers): ModuleList(
      (0): MimiConv1d(
        (conv): Conv1d(1, 64, kernel_size=(7,), stride=(1,))
      )
      (1): MimiResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): MimiConv1d(
            (conv): Conv1d(64, 32, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): MimiConv1d(
            (conv): Conv1d(32, 64, kernel_size=(1,), stride=(1,))
          )
        )
        (shortcut): Identity()
      )
      (2): ELU(alpha=1.0)
      (3): MimiConv1d(
        (conv): Conv1d(64, 128, kernel_size=(8,), stride=(4,))
      )
      (4): MimiResnetBlock(
        (block): ModuleList(
          (0): ELU(alpha=1.0)
          (1): MimiConv1d(
            (conv): Conv1d(128, 64, kernel_size=(3,), stride=(1,))
          )
          (2): ELU(alpha=1.0)
          (3): MimiConv1d(
            (conv): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
          )
        )
        (s

### 2.2 Load The Data

In [10]:
import pandas as pd
from datasets import Dataset

# Path to the dataset after git clone
dataset_path = os.path.join("Elise", "data", "train-00000-of-00001.parquet")

df = pd.read_parquet(dataset_path)
ds = {"train": Dataset.from_pandas(df)}
print(f"Dataset structure: {ds}")

# Ensure the audio is 24kHz (CSM requirement)
# Check if 'audio' column exists, otherwise look for the correct audio column
audio_column = "audio"
if audio_column in ds["train"].column_names:
    ds["train"] = ds["train"].cast_column(audio_column, Audio(sampling_rate=24000))
else:
    print(f"Warning: Column '{audio_column}' not found. Available columns: {ds['train'].column_names}")
    # Try to identify the audio column if it has a different name
    # You might need to adjust this based on your dataset structure

Dataset structure: {'train': Dataset({
    features: ['audio', 'text'],
    num_rows: 1195
})}


In [11]:
def prepare_conversation_batch(batch_size=4, offset=0):
    """Prepare a batch of conversations from the dataset"""
    batch_indices = list(range(offset, min(offset + batch_size, len(ds["train"]))))
    conversation = []

    # Use a stringified integer for the speaker ID (e.g., "0")
    speaker_id = "0"  # Changed from "Elise" to "0"

    for idx in batch_indices:
        example = ds["train"][idx]

        # Extract text and audio from the dataset
        text = example.get("text", "")

        # Handle audio data correctly based on dataset structure
        if "audio" in example and isinstance(example["audio"], dict) and "array" in example["audio"]:
            audio_data = example["audio"]["array"]
        elif "audio" in example:
            # If audio is directly accessible
            audio_data = example["audio"]
        else:
            # Try to find audio under a different key
            print(f"Warning: Audio not found in example. Available keys: {example.keys()}")
            continue

        conversation.append({
            "role": speaker_id,  # Now using a stringified integer
            "content": [
                {"type": "text", "text": text},
                {"type": "audio", "path": audio_data}
            ],
        })

    return conversation

## 3. Modeling

In [ ]:
# Add these imports at the top
from peft import LoraConfig, get_peft_model, TaskType
from peft import PeftModel, PeftConfig

# Replace the model loading section
model_id = "sesame/csm-1b"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load the model and processor
processor = AutoProcessor.from_pretrained(model_id)
base_model = CsmForConditionalGeneration.from_pretrained(model_id, device_map=device)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # or TaskType.CAUSAL_LM depending on the model
    inference_mode=False,
    r=16,  # rank - lower values (8-32) for more conservative fine-tuning
    lora_alpha=32,  # scaling parameter
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # target attention and MLP layers
    bias="none",
)

# Apply LoRA to the model
model = get_peft_model(base_model, lora_config)
model.train()
model.base_model.codec_model.eval()  # Keep codec model in eval mode

# Print trainable parameters
model.print_trainable_parameters()

In [ ]:
# Training configurations
output_dir = "./csm_elise_lora_model"
num_train_epochs = 5
per_device_train_batch_size = 2  # CSM models can be memory intensive
gradient_accumulation_steps = 2
learning_rate = 1e-4
warmup_steps = 100
logging_steps = 10
save_steps = 500
max_steps = 1000  # Adjust based on dataset size
weight_decay = 0.01  # Add weight decay
max_grad_norm = 1.0  # Add gradient clipping

# Initialize training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    warmup_steps=warmup_steps,
    logging_steps=logging_steps,
    save_steps=save_steps,
    max_steps=max_steps,
    fp16=True,  # Use mixed precision training
    remove_unused_columns=False,
    report_to="tensorboard",
)

# Custom training loop (as alternative to using Trainer)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,
                                             start_factor=1.0,
                                             end_factor=0.0,
                                             total_iters=max_steps)

# Training loop
model.train()
total_batches = min(max_steps, len(ds["train"]) // per_device_train_batch_size)

for step in tqdm(range(total_batches)):
    batch_offset = step * per_device_train_batch_size
    conversation = prepare_conversation_batch(per_device_train_batch_size, batch_offset)

    # Skip empty conversations
    if not conversation:
        continue

    # Process the conversation batch
    try:
        inputs = processor.apply_chat_template(
            conversation,
            tokenize=True,
            return_dict=True,
            output_labels=True,
        ).to(device)

        # Forward pass
        outputs = model(**inputs)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Add gradient clipping here
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        # Optimizer step with gradient accumulation
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        # Log progress
        if step % logging_steps == 0:
            print(f"Step {step}, Loss: {loss.item()}")

        # Save checkpoint
        if step % save_steps == 0 and step > 0:
            model.save_pretrained(f"{output_dir}/checkpoint-{step}")
            processor.save_pretrained(f"{output_dir}/checkpoint-{step}")

    except Exception as e:
        print(f"Error in batch at offset {batch_offset}: {e}")
        continue

  0%|          | 1/1195 [00:16<5:24:44, 16.32s/it]

Step 0, Loss: 5.28347635269165


  1%|          | 11/1195 [00:19<09:07,  2.16it/s]

Step 10, Loss: 6.476818561553955


  2%|▏         | 21/1195 [00:22<06:44,  2.90it/s]

...

 99%|█████████▉| 1181/1195 [07:31<00:04,  3.05it/s]

Step 1180, Loss: 4.829275131225586


100%|█████████▉| 1191/1195 [07:34<00:01,  3.05it/s]

Step 1190, Loss: 5.5672287940979


100%|██████████| 1195/1195 [07:35<00:00,  2.62it/s]


Now save the trained model in local directory.

In [ ]:
# Update the model saving section
# Save the LoRA adapter (much smaller than full model)
model.save_pretrained(f"{output_dir}/final")
processor.save_pretrained(f"{output_dir}/final")

# Also save the base model reference for easier loading later
with open(f"{output_dir}/final/base_model_name.txt", "w") as f:
    f.write(model_id)

[]

## 4. Inference

Try the model with some sample inputs to see how it performs.

In [ ]:
# Modified load_peft_model_for_inference function
def load_peft_model_for_inference(adapter_path, base_model_id=None):
    if base_model_id is None:
        try:
            with open(f"{adapter_path}/base_model_name.txt", "r") as f:
                base_model_id = f.read().strip()
        except:
            base_model_id = "sesame/csm-1b"  # fallback
    
    # Load base model
    base_model = CsmForConditionalGeneration.from_pretrained(base_model_id, device_map=device)
    
    # Load and merge PEFT model
    model = PeftModel.from_pretrained(base_model, adapter_path)
    # This is the key step - merge weights to remove adapter overhead
    merged_model = model.merge_and_unload()
    return merged_model

In [ ]:
from transformers import BitsAndBytesConfig

def load_optimized_peft_model(adapter_path, base_model_id=None):
    # Similar setup as before
    base_model = CsmForConditionalGeneration.from_pretrained(
        base_model_id, 
        device_map=device,
        torch_dtype=torch.float16,  # Use FP16 for faster inference
    )
    
    model = PeftModel.from_pretrained(base_model, adapter_path)
    merged_model = model.merge_and_unload()
    
    # Now convert to half precision after merging
    merged_model = merged_model.half()
    return merged_model

In [ ]:
# Set these optimization parameters for generation
def optimize_generation_config(model):
    model.generation_config.max_length = 256  # Or appropriate value for your needs
    model.generation_config.use_cache = True
    model.generation_config.cache_implementation = "static"
    
    # If model has a depth decoder
    if hasattr(model, "depth_decoder"):
        model.depth_decoder.generation_config.cache_implementation = "static"
    
    return model

# Use after loading the model
model = load_peft_model_for_inference(f"{output_dir}/final")
model = optimize_generation_config(model)

### 4.1 Generate a Sentence

In [15]:
from IPython.display import Audio, display
import soundfile as sf

os.makedirs("inference", exist_ok=True)

# another equivalent way to prepare the inputs
conversation = [
    {"role": "0", "content": [{"type": "text", "text": "The past is just a story we tell ourselves. You're making me blush <giggles>. I love you! <giggles>"}]},
]

inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True)

# Move tensor from GPU to CPU before converting to numpy
audio_cpu = audio[0].to(torch.float32).cpu().numpy()  # This fixes the TypeError

# Write to file and play
output_file = "inference/output_nc.wav"
sf.write(output_file, audio_cpu, 24000)
display(Audio(output_file))

In [19]:
conversation = [
    {"role": "0", "content": [{"type": "text", "text": "You're making me blush! I'm climaxing! Coming! Ohh! That's so good! More. Ohh. More. Go harder!"}]},
]

inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True)

# Move tensor from GPU to CPU before converting to numpy
audio_cpu = audio[0].to(torch.float32).cpu().numpy()  # This fixes the TypeError

# Write to file and play
output_file = "inference/output_2.wav"
sf.write(output_file, audio_cpu, 24000)
display(Audio(output_file))

### 4.2 Sound With Context

Audio influence by giving a context to the model.

In [23]:
conversation = []

# 1. context
# This code takes the first 4 examples from the dataset to provide context
for text, audio in zip(ds["train"][:4]["text"], ds["train"][:4]["audio"]):
    conversation.append(
        {
            "role": f"0",
            "content": [{"type": "text", "text": text}, {"type": "audio", "path": audio["array"]}],
        }
    )

# 2. text prompt
conversation.append({"role": "0", "content": [{"type": "text", "text": ds["train"][4]["text"]}]})

inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True)

# Move tensor from GPU to CPU before converting to numpy
audio_cpu = audio[0].to(torch.float32).cpu().numpy()  # This fixes the TypeError

# Write to file and play
output_file = "inference/output_wc.wav"
sf.write(output_file, audio_cpu, 24000)
display(Audio(output_file))

### 4.3 Batch Inference

Different audio characteristics:
- Conversation 1: Audio influenced by the voice context from ds[0]["audio"]
- Conversation 2: Audio generated without voice context (using default model voice)

In [25]:
conversation = [
    [
        {
            "role": f"0",
            "content": [
                {"type": "text", "text": ds["train"][0]["text"]},
                {"type": "audio", "path": ds["train"][0]["audio"]["array"]},
            ],
        },
        {
            "role": f"0",
            "content": [
                {"type": "text", "text": ds["train"][1]["text"]},
            ],
        },
    ],
    [
        {
            "role": f"0",
            "content": [
                {"type": "text", "text": ds["train"][0]["text"]},
            ],
        }
    ],
]

inputs = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

# infer the model
audio = model.generate(**inputs, output_audio=True)

# Process all batch outputs
for i, audio_output in enumerate(audio):
    audio_cpu = audio_output.to(torch.float32).cpu().numpy()
    output_file = f"inference/output_batch_{i}.wav"
    sf.write(output_file, audio_cpu, 24000)
    print(f"Saved: {output_file}")
    display(Audio(output_file))

Saved: inference/output_batch_0.wav


Saved: inference/output_batch_1.wav


In [28]:
# use static cache, enabling automatically torch compile with fullgraph and reduce-overhead
model.generation_config.max_length = 512 # big enough to avoid recompilation
model.generation_config.max_new_tokens = None # would take precedence over max_length
model.generation_config.cache_implementation = "static"
model.depth_decoder.generation_config.cache_implementation = "static"

# generation kwargs
gen_kwargs = {
    "do_sample": False,
    "depth_decoder_do_sample": False,
    "temperature": 1.0,
    "depth_decoder_temperature": 1.0,
}

# Define a timing decorator
class TimerContext:
    def __init__(self, name="Execution"):
        self.name = name
        self.start_event = None
        self.end_event = None

    def __enter__(self):
        # Use CUDA events for more accurate GPU timing
        self.start_event = torch.cuda.Event(enable_timing=True)
        self.end_event = torch.cuda.Event(enable_timing=True)
        self.start_event.record()
        return self

    def __exit__(self, *args):
        self.end_event.record()
        torch.cuda.synchronize()
        elapsed_time = self.start_event.elapsed_time(self.end_event) / 1000.0
        print(f"{self.name} time: {elapsed_time:.4f} seconds")

conversation = [
    {
        "role": f"0",
        "content": [
            {"type": "text", "text": ds["train"][0]["text"]},
            {"type": "audio", "path": ds["train"][0]["audio"]["array"]},
        ],
    },
    {
        "role": f"0",
        "content": [
            {"type": "text", "text": ds["train"][1]["text"]},
            {"type": "audio", "path": ds["train"][1]["audio"]["array"]},
        ],
    },
    {
        "role": f"0",
        "content": [
            {"type": "text", "text": ds["train"][2]["text"]},
        ],
    },
]

padded_inputs_1 = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

print("\n" + "="*50)
print("First generation - compiling and recording CUDA graphs...")
with TimerContext("First generation"):
    _ = model.generate(**padded_inputs_1, **gen_kwargs)
print("="*50)

print("\n" + "="*50)
print("Second generation - fast !!!")
with TimerContext("Second generation"):
    _ = model.generate(**padded_inputs_1, **gen_kwargs)
print("="*50)

# now with different inputs
conversation = [
    {
        "role": f"{0}",
        "content": [
            {"type": "text", "text": ds["train"][2]["text"]},
            {"type": "audio", "path": ds["train"][2]["audio"]["array"]},
        ],
    },
    {
        "role": f"{0}",
        "content": [
            {"type": "text", "text": ds["train"][3]["text"]},
            {"type": "audio", "path": ds["train"][3]["audio"]["array"]},
        ],
    },
    {
        "role": f"{0}",
        "content": [
            {"type": "text", "text": ds["train"][4]["text"]},
        ],
    },
]
padded_inputs_2 = processor.apply_chat_template(
    conversation,
    tokenize=True,
    return_dict=True,
).to(device)

print("\n" + "="*50)
print("Generation with other inputs!")
with TimerContext("Generation with different inputs"):
    _ = model.generate(**padded_inputs_2, **gen_kwargs)
print("="*50)


First generation - compiling and recording CUDA graphs...
First generation time: 12.0582 seconds

Second generation - fast !!!
Second generation time: 11.9195 seconds

Generation with other inputs!
Generation with different inputs time: 4.5732 seconds


## Save To Hugging Face Hub

Finally, save the model to the Hugging Face Hub for future use or sharing.

In [29]:
from huggingface_hub import HfApi

# Define your model repository name (create this on the HF website first)
model_name = "keanteng/sesame-csm-elise"  # Replace with your desired repo name

# Alternatively, push an already saved model directory
api = HfApi()
api.upload_folder(
    folder_path=f"{output_dir}/final",
    repo_id=model_name,
    repo_type="model"
)

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/keanteng/sesame-csm-elise/commit/4b2e536dffe18bd570116040723c40134a1fc66d', commit_message='Upload folder using huggingface_hub', commit_description='', oid='4b2e536dffe18bd570116040723c40134a1fc66d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/keanteng/sesame-csm-elise', endpoint='https://huggingface.co', repo_type='model', repo_id='keanteng/sesame-csm-elise'), pr_revision=None, pr_num=None)

In [30]:
# Add a model card with description
with open(f"{output_dir}/README.md", "w") as f:
    f.write("""
---
license: agpl-3.0
datasets:
- MrDragonFox/Elise
language:
- en
base_model:
- sesame/csm-1b
pipeline_tag: text-to-speech
library_name: transformers
tags:
- generative-ai
---
# CSM Elise Voice Model

This model is a fine-tuned version of [sesame/csm-1b](https://huggingface.co/sesame/csm-1b) using the [Elise dataset](https://huggingface.co/datasets/MrDragonFox/Elise). There are sample outputs files in the repository.

## Model Details
- **Base Model**: sesame/csm-1b
- **Training Data**: MrDragonFox/Elise dataset
- **Fine-tuning Approach**: Voice cloning through conditional speech generation
- **Voice Characteristics**: [Describe voice qualities]
- **Training Parameters**:
  - Learning Rate: 5e-5
  - Epochs: 3
  - Batch Size: 1 with gradient accumulation steps of 4
""")

In [31]:
api.upload_file(
    path_or_fileobj=f"{output_dir}/README.md",
    path_in_repo="README.md",
    repo_id=model_name,
    repo_type="model"
)

CommitInfo(commit_url='https://huggingface.co/keanteng/sesame-csm-elise/commit/62bf83ae63c89cab0b16b27a7ef3ff944d7454f4', commit_message='Upload README.md with huggingface_hub', commit_description='', oid='62bf83ae63c89cab0b16b27a7ef3ff944d7454f4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/keanteng/sesame-csm-elise', endpoint='https://huggingface.co', repo_type='model', repo_id='keanteng/sesame-csm-elise'), pr_revision=None, pr_num=None)

In [36]:
api.upload_folder(
    folder_path=f"inference",
    repo_id=f"{model_name}",
    repo_type="model",
    path_in_repo="inference"
)

output_nc.wav:   0%|          | 0.00/388k [00:00<?, ?B/s]

output_batch_0.wav:   0%|          | 0.00/480k [00:00<?, ?B/s]

output_batch_1.wav:   0%|          | 0.00/480k [00:00<?, ?B/s]

output_wc.wav:   0%|          | 0.00/219k [00:00<?, ?B/s]

output_2.wav:   0%|          | 0.00/480k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/keanteng/sesame-csm-elise/commit/52909e34ac4c8070186015d104dec45935628ed5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='52909e34ac4c8070186015d104dec45935628ed5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/keanteng/sesame-csm-elise', endpoint='https://huggingface.co', repo_type='model', repo_id='keanteng/sesame-csm-elise'), pr_revision=None, pr_num=None)

## 5. Other

In [37]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
31.252 GB of memory reserved.
